# System ID Introduction

We'll be taking a look through a few common methods of examining the input and output signals of a system using Python.


Most of this will be done using the "signal" toolkit from SciPy, with help from Numpy.


First, let's import all of the necessary modules

In [37]:
from scipy import signal
import numpy as np
from matplotlib import pyplot as plt
# import ipympl # This is nice for better interactive plots
import pandas as pd
%matplotlib qt

First, let's create a few "standard" waveforms -- A step, a unit impulse, a sine wave, and a chirp.

In [38]:
fs = 1000
ts = np.arange(-1,60,1/fs)      # times

impulse = np.zeros(ts.shape)    # impulse is zero...
impulse[fs] = 1               # except at zero

step = np.zeros(ts.shape)       # step is zero before zero
step[fs:] = 1                 # and 1 after

sine = np.sin(ts*20*(2*np.pi))               # sine wave with freq. of 20 hz

chirp = signal.chirp(ts,1,50,40, method='logarithmic')

rand = np.random.random(size=ts.shape)

Now let's plot them all to see what we're looking at

In [39]:
fig_base,ax_base = plt.subplots(nrows=5, sharex=True)

ax_base[0].plot(ts,impulse, label='Impulse')
ax_base[1].plot(ts,step, label='Step')
ax_base[2].plot(ts,sine, label='Sine')
ax_base[3].plot(ts,chirp, label='Chirp')
ax_base[4].plot(ts,rand, label='Rand')

for ii in np.arange(4):
    ax_base[ii].spines[:].set_visible(False)
    ax_base[ii].set_ylim([-1.1,1.1])
    ax_base[ii].set_xticks(ticks=[])


fig_base.tight_layout()

### Power Spectra

The power spectra looks at the power density of the signal at different frequencies. We will expect to see some interesting stuff from the delta and step functions

In [40]:
# f_impulse,p_impulse = signal.welch(impulse, fs, scaling='spectrum')
# f_step,p_step = signal.welch(step, fs, scaling='spectrum')
# f_sine,p_sine = signal.welch(sine, fs, scaling='spectrum')
# f_chirp,p_chirp = signal.welch(chirp, fs, scaling='spectrum')
# f_rand,p_rand = signal.welch(rand, fs, scaling='spectrum')



f_impulse,p_impulse = signal.welch(impulse, fs, scaling='spectrum', nperseg=1024)
f_step,p_step = signal.welch(step, fs, scaling='spectrum', nperseg=1024)
f_sine,p_sine = signal.welch(sine, fs, scaling='spectrum', nperseg=1024)
f_chirp,p_chirp = signal.welch(chirp, fs, scaling='spectrum', nperseg=1024)
f_rand,p_rand = signal.welch(rand, fs, scaling='spectrum', nperseg=1024)

In [41]:
fig_welch,ax_welch = plt.subplots(nrows=5, sharex=True)

ax_welch[0].plot(f_impulse, p_impulse, label='impulse')
ax_welch[0].set_title('impulse')
ax_welch[1].plot(f_step, p_step, label='step')
ax_welch[1].set_title('step')
ax_welch[2].plot(f_sine, p_sine, label='sine')
ax_welch[2].set_title('sine')
ax_welch[3].plot(f_chirp, p_chirp, label='chirp')
ax_welch[3].set_title('chirp')
ax_welch[4].plot(f_rand, p_rand, label='rand')
ax_welch[4].set_title('rand')

for ii in np.arange(5):
    ax_welch[ii].spines[:].set_visible(False)
    # ax_welch[ii].set_ylim([-1.1,1.1])
    ax_welch[ii].set_xticks(ticks=[])
    ax_welch[ii].set_xscale('log')

fig_welch.tight_layout(h_pad=1)

### Impulse response

We're going to make a 51 sample impulse, and convolve it with the signals to see how each responds

In [42]:
imp_51 = np.zeros([51,])
imp_51[25] = 1

impulse_imp = signal.convolve(impulse,imp_51)
step_imp = signal.convolve(step,imp_51)
sine_imp = signal.convolve(sine,imp_51)
chirp_imp = signal.convolve(chirp,imp_51)
rand_imp = signal.convolve(rand,imp_51)

In [43]:
fig_imp,ax_imp = plt.subplots(nrows=5, sharex=True)

ax_imp[0].plot(impulse_imp)
ax_imp[0].set_title('impulse')
ax_imp[1].plot(step_imp)
ax_imp[1].set_title('step')
ax_imp[2].plot(sine_imp)
ax_imp[2].set_title('sine')
ax_imp[3].plot(chirp_imp)
ax_imp[3].set_title('chirp')
ax_imp[4].plot(rand_imp)
ax_imp[4].set_title('rand')


for ii in np.arange(5):
    ax_imp[ii].spines[:].set_visible(False)
    # ax_imp[ii].set_ylim([-1.1,1.1])
    ax_imp[ii].set_xticks(ticks=[])

fig_imp.tight_layout(h_pad=1)

### Filtering

Let's try filtering with a couple different filters

In [44]:
sos = signal.butter(4,np.array([10, 30]), 'bandpass', fs=fs, output='sos')
# sos = signal.butter(4,np.array([10]), 'lowpass', fs=fs)
# sos = signal.butter(4,np.array([30]), 'highpass', fs=fs)

impulse_filt_1 = signal.sosfilt(sos,impulse)
step_filt_1 = signal.sosfilt(sos,step)
sine_filt_1 = signal.sosfilt(sos,sine)
chirp_filt_1 = signal.sosfilt(sos,chirp)
rand_filt_1 = signal.sosfilt(sos,rand)




In [45]:
fig_filt_1, ax_filt_1 = plt.subplots(nrows=5, sharex=True)


ax_filt_1[0].plot(ts,impulse_filt_1)
ax_filt_1[0].set_title('impulse')
ax_filt_1[1].plot(ts,step_filt_1)
ax_filt_1[1].set_title('step')
ax_filt_1[2].plot(ts,sine_filt_1)
ax_filt_1[2].set_title('sine')
ax_filt_1[3].plot(ts,chirp_filt_1)
ax_filt_1[3].set_title('chirp')
ax_filt_1[4].plot(ts,rand_filt_1)
ax_filt_1[4].set_title('rand')

for ii in np.arange(5):
    ax_filt_1[ii].spines[:].set_visible(False)
    # ax_filt_1[ii].set_ylim([-1.1,1.1])
    ax_filt_1[ii].set_xticks(ticks=[])

fig_filt_1.tight_layout(h_pad=1)